### 전처리 

In [31]:
import pymongo as mg
import pandas as pd

In [32]:
client = mg.MongoClient(host='mongodb://localhost:27017')

In [36]:
database = client['db_NHIS']
collection = database['merged_collection6']

In [37]:
cursor = collection.find({})
list_BND_M20 = list(cursor)

In [38]:
df_BND_M20 = pd.DataFrame(list_BND_M20)
df_BND_M20[:2]

,_id,RN_INDI,BTH_YYYY,DTH_YYYYMM,COD1,COD2,RN_KEY,RN_INST,MDCARE_STRT_DT,FORM_CD,...,OPRTN_YN,MDCARE_DD_CNT,VSHSP_DD_CNT,TOT_PRSC_DD_CNT,MCARE_RSLT_TYPE,EDC_ADD_RT,ED_RC_TOT_AMT,EDC_SBA,EDC_INSUR_BRDN_AMT,STD_YYYY
0,64e5d26e83f3c689386edb41,294364,1921LE,200707.0,T08-T14,W00-W19,2002080298703,26452,20020830,3,...,0,1,1,3,5.0,0.15,10590,3000,7590,2002
1,64e5d26e83f3c689386edb42,294364,1921LE,200707.0,T08-T14,W00-W19,2002090466395,63715,20020910,3,...,0,1,1,2,5.0,0.15,16520,4950,11570,2002


In [39]:
#세자리씩 잘라서 새로운 컬럼 생성, 머신러닝 위해서 숫자화
#index를 code에 넣는 게 가장 쉬운 방식 
df_BND_M20['SICK_SYM1'].value_counts()

J209     8244
I10      4629
I109     3275
F_       3203
N_       3177
         ... 
H700        1
E058        1
M5397       1
T18         1
P289        1
Name: SICK_SYM1, Length: 3414, dtype: int64

In [40]:
#뒷 세자리만 남기는 방식 
len('F_'), len('J209'), 'J209'[:3], str('T18')[:3]

(2, 4, 'J20', 'T18')

In [42]:
def convertSICK_SYM(sick_sym) : 
    if len(sick_sym) < 3:
        return None
    else :
        return sick_sym[:3]


In [43]:
 convertSICK_SYM('J209'),  convertSICK_SYM('F_')

('J20', None)

In [46]:
df_BND_M20['SICK_SYM1_3'] = df_BND_M20['SICK_SYM1'].apply( convertSICK_SYM)

In [47]:
df_BND_M20['SICK_SYM1_3'].value_counts()

J20    8917
I10    8427
J03    4187
M54    3904
J06    3460
       ... 
G97       1
K01       1
S28       1
I72       1
P28       1
Name: SICK_SYM1_3, Length: 747, dtype: int64

In [48]:
df_BND_M20_dropna = df_BND_M20.dropna()
df_BND_M20_dropna.isnull().sum() 

_id                   0
RN_INDI               0
BTH_YYYY              0
DTH_YYYYMM            0
COD1                  0
COD2                  0
RN_KEY                0
RN_INST               0
MDCARE_STRT_DT        0
FORM_CD               0
MCARE_SUBJ_CD         0
SICK_SYM1             0
OFIJ_TYPE             0
OPRTN_YN              0
MDCARE_DD_CNT         0
VSHSP_DD_CNT          0
TOT_PRSC_DD_CNT       0
MCARE_RSLT_TYPE       0
EDC_ADD_RT            0
ED_RC_TOT_AMT         0
EDC_SBA               0
EDC_INSUR_BRDN_AMT    0
STD_YYYY              0
SICK_SYM1_3           0
dtype: int64

In [57]:
df_BND_M20_dropna[:2]

,_id,RN_INDI,BTH_YYYY,DTH_YYYYMM,COD1,COD2,RN_KEY,RN_INST,MDCARE_STRT_DT,FORM_CD,...,MDCARE_DD_CNT,VSHSP_DD_CNT,TOT_PRSC_DD_CNT,MCARE_RSLT_TYPE,EDC_ADD_RT,ED_RC_TOT_AMT,EDC_SBA,EDC_INSUR_BRDN_AMT,STD_YYYY,SICK_SYM1_3
0,64e5d26e83f3c689386edb41,294364,1921LE,200707.0,T08-T14,W00-W19,2002080298703,26452,20020830,3,...,1,1,3,5.0,0.15,10590,3000,7590,2002,L02
3,64e5d26e83f3c689386edb44,294364,1921LE,200707.0,T08-T14,W00-W19,2002020488631,23506,20020208,3,...,1,1,3,5.0,0.15,12920,3000,9920,2002,K29


In [60]:
target_list = df_BND_M20_dropna['SICK_SYM1_3'].value_counts()
target_list 

J20    68
E11    56
I10    50
J30    35
I42    32
       ..
R81     1
H36     1
K12     1
E03     1
H00     1
Name: SICK_SYM1_3, Length: 141, dtype: int64

In [63]:
target_list = df_BND_M20_dropna['SICK_SYM1_3'].value_counts().index.to_list() 
#인덱스를 리스트로 만들어

In [67]:
target_list.index('E11') 

1

In [65]:
target_list.index('J20') 

0

In [73]:
def applyIndexNumber(SICK_SYM1_3):
    indexNumber = target_list.index(SICK_SYM1_3)
    return indexNumber 

In [75]:
df_BND_M20_dropna['SICK_SYM1_3_target'] = df_BND_M20_dropna['SICK_SYM1_3'].apply(applyIndexNumber)

C:\Users\5-21\AppData\Local\Temp\ipykernel_37236\3294566143.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_BND_M20_dropna['SICK_SYM1_3_target'] = df_BND_M20_dropna['SICK_SYM1_3'].apply(applyIndexNumber)


In [77]:
df_BND_M20_dropna['SICK_SYM1_3_target'][:2]

0    83
3     8
Name: SICK_SYM1_3_target, dtype: int64